In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("pyspark-basics").getOrCreate()

df = spark.read.csv("gs://bucket-name/stackoverflowposts.csv",header=True)

df_subset = df.select(["id","tags","score","view_count","creation_date"])

In [ ]:
df_subset.show(4,False)

In [ ]:
df_subset.filter(df_subset.tags.like('python%')).show(5,False)

In [ ]:
from pyspark.sql import functions as F 

In [ ]:
df_tagged = df_subset \
        .withColumn(
        'post_type',
        F.when(F.col('tags').like('python%'),'python') \
        .otherwise(
            F.when(F.col('tags').like('mysql%'),'mysql')
            .otherwise("others")
        )
)

In [ ]:
df_tagged.show(10,False)

In [ ]:
from pyspark.sql.types import IntegerType

df_tagged = df_tagged.withColumn("view_count",df_tagged["view_count"].cast(IntegerType()))

In [ ]:
df_tagged.printSchema()

In [ ]:
df_tagged.groupBy("post_type") \
.agg(
    F.sum("view_count").alias("total_views"),
    F.count("id").alias("total_row_count")
).show(10,False)